This notebook will form the evaluation on which model to select for my PhishGuard tool.
Consider the data that is to be trained I am considering the following models:
- K - Nearest Neighbours (KNN)
- Random Forest Classifier (RFC)
- Gradient Boost Classifier (GBC)
- Support Vector Machines (SVM)
- Logistic Regression

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
import numpy as np
import seaborn as sns



In [25]:
df = shuffle(pd.read_csv("./datasets/dataset.csv"))

df.head
df.info

#sns.pairplot(df,palette='Dark2')

<bound method DataFrame.info of        index  having_IPhaving_IP_Address  URLURL_Length  Shortining_Service   
4635    4636                           1             -1                   1  \
10638  10639                          -1             -1                   1   
10380  10381                          -1             -1                   1   
1039    1040                           1             -1                   1   
6262    6263                          -1             -1                   1   
...      ...                         ...            ...                 ...   
2625    2626                           1             -1                   1   
10126  10127                          -1             -1                   1   
4948    4949                           1             -1                   1   
3956    3957                           1             -1                   1   
6947    6948                           1             -1                   1   

       having_At_Sy

In [26]:
features = [
    "having_IPhaving_IP_Address", # URL contains IP
    "URLURL_Length", # Length of the URL
    "having_At_Symbol", # URL contains @
    "Shortining_Service", # Long URL shorted eg. bit.ly...
    "double_slash_redirecting", # URL contains redirect to another site
    "having_Sub_Domain", # www.aus.go.au
    "SSLfinal_State", # URL uses HTTPS
    "Domain_registeration_length", # URL domain is registed for short period 
    "HTTPS_token", # URL contains http://https
    "Google_Index", # Can be find via google or not
    "Page_Rank", # Measures how important a page is on the internet 0 or 1
    "port", # Scammers open all ports on there server to allow all services
    "Favicon", # If the favicon is loaded from a domain other than that shown in the address bar - phish
    "Abnormal_URL", # Fetch by WHOIS database - API
    "age_of_domain", # Check current domain age
    "DNSRecord"
    ]


df.dropna(inplace=True)

print(df.head)


X = df[features]
Y = df["Result"]

X.dropna()
Y.dropna()

X.drop_duplicates()
Y.drop_duplicates()

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, shuffle=True)


<bound method NDFrame.head of        index  having_IPhaving_IP_Address  URLURL_Length  Shortining_Service   
4635    4636                           1             -1                   1  \
10638  10639                          -1             -1                   1   
10380  10381                          -1             -1                   1   
1039    1040                           1             -1                   1   
6262    6263                          -1             -1                   1   
...      ...                         ...            ...                 ...   
2625    2626                           1             -1                   1   
10126  10127                          -1             -1                   1   
4948    4949                           1             -1                   1   
3956    3957                           1             -1                   1   
6947    6948                           1             -1                   1   

       having_At_Symb

In [27]:
default_knn = KNeighborsClassifier()
default_knn.fit(X_train,y_train)
y_pred_knn_default  = default_knn.predict(X_test)

In [28]:
estimator = KNeighborsClassifier(algorithm='auto')

parameters_KNN = {
    'n_neighbors': (1,10, 1),
    'leaf_size': (20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev')
}
grid_search_KNN = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_KNN,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5
)

In [29]:
KNN_1=grid_search_KNN.fit(X_train, y_train)
y_pred_KNN1 =KNN_1.predict(X_test)
KNN_2=grid_search_KNN.fit(X_train, y_train)
y_pred_KNN2 =KNN_2.predict(X_test)
KNN_3=grid_search_KNN.fit(X_train, y_train)
y_pred_KNN3 =KNN_3.predict(X_test)
# display the best parameters for KNN classifier
print(grid_search_KNN.best_params_ ) 
# mean cross validated score of the best estimator
print('Best Score - KNN:', grid_search_KNN.best_score_ )

{'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 10, 'p': 1, 'weights': 'distance'}
Best Score - KNN: 0.9017422349206674


In [30]:
print('Accuracy Score - KNN Default:', metrics.accuracy_score(y_test, y_pred_knn_default))  
print('Accuracy Score - KNN_1:', metrics.accuracy_score(y_test, y_pred_KNN1)) 
print('Accuracy Score - KNN_2:', metrics.accuracy_score(y_test, y_pred_KNN2)) 
print('Accuracy Score - KNN _3: ', metrics.accuracy_score(y_test, y_pred_KNN3)) 

Accuracy Score - KNN Default: 0.8919041157847128
Accuracy Score - KNN_1: 0.9154228855721394
Accuracy Score - KNN_2: 0.9154228855721394
Accuracy Score - KNN _3:  0.9154228855721394


In [31]:
print(KNN_1.best_params_)

{'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 10, 'p': 1, 'weights': 'distance'}



KNN Evaluation Complete
===

In [32]:
from sklearn.ensemble import RandomForestRegressor

rfr_default = RandomForestRegressor(random_state=42)
print(rfr_default.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [33]:
'''
n_estimators = number of trees in random forest
max_features = number of features consider in each split
max_depth = maximum number of levels within the tree
min_samples_split = samples required at each split node
min_samples_leaf = minimum number if leafs at each node
boostrap = selecting training method at each level
'''
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

In [34]:
parameters_RFR = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
    }

grid_search_RFR = GridSearchCV(
    estimator=rfr_default,
    param_grid=parameters_RFR,
    scoring='f1',
    n_jobs=-1,
    cv=5,
    )


In [35]:
RFR_1 = grid_search_RFR.fit(X_train, y_train)
y_pred_RFR1 = RFR_1.predict(X_test)
RFR_2 = grid_search_RFR.fit(X_train, y_train)
y_pred_RFR2 = RFR_2.predict(X_test)
RFR_3 = grid_search_RFR.fit(X_train, y_train)
y_pred_RFR3 = KNN_3.predict(X_test)
print(grid_search_RFR.best_params_ ) 
print('Best Score - RFR:', grid_search_RFR.best_score_ )

/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter a

KeyboardInterrupt: 

Issue with KNN model moving to SVM
===

In [36]:
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix  

# Different kernels allow different approaches to transforming data into the required form
kernels = ["poly", "rbf", "sigmoid", "linear"]
for i in range(len(kernels)):
    # new set every iteration
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, shuffle=True)
    if i == 0: svm_classifier = SVC(kernel=kernels[i],degree=2,gamma="auto")
    else: svm_classifier = SVC(kernel=kernels[i],gamma="auto")
    svm_classifier.fit(X_train, y_train)
    y_pred = svm_classifier.predict(X_test)
    print(kernels[i])
    print(classification_report(y_test, y_pred))




poly
              precision    recall  f1-score   support

          -1       0.87      0.81      0.84       954
           1       0.86      0.91      0.88      1257

    accuracy                           0.86      2211
   macro avg       0.86      0.86      0.86      2211
weighted avg       0.86      0.86      0.86      2211

rbf
              precision    recall  f1-score   support

          -1       0.88      0.85      0.87       980
           1       0.88      0.91      0.90      1231

    accuracy                           0.88      2211
   macro avg       0.88      0.88      0.88      2211
weighted avg       0.88      0.88      0.88      2211

sigmoid
              precision    recall  f1-score   support

          -1       0.69      0.70      0.70       988
           1       0.76      0.75      0.75      1223

    accuracy                           0.73      2211
   macro avg       0.72      0.72      0.72      2211
weighted avg       0.73      0.73      0.73      2211

li

In [37]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
lin_pred = linear_svc.predict(X_test)

params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['hinge', 'squared_hinge'],
    'dual': [True, False],
    'tol': [1e-3, 1e-4, 1e-5],
    'max_iter': [1000, 2000, 3000],
    'class_weight': [None, 'balanced']
}
svc_grid_search = GridSearchCV(estimator=linear_svc, param_grid=params, 
                           cv=5, n_jobs=-1, verbose=2)
svc_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits
[CV] END C=0.01, class_weight=None, dual=True, loss=hinge, max_iter=1000, penalty=l1, tol=0.0001; total time=   0.0s[CV] END C=0.01, class_weight=None, dual=True, loss=hinge, max_iter=1000, penalty=l1, tol=0.001; total time=   0.0s

[CV] END C=0.01, class_weight=None, dual=True, loss=hinge, max_iter=1000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=0.01, class_weight=None, dual=True, loss=hinge, max_iter=1000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=0.01, class_weight=None, dual=True, loss=hinge, max_iter=1000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=0.01, class_weight=None, dual=True, loss=hinge, max_iter=1000, penalty=l1, tol=0.0001; total time=   0.0s
[CV] END C=0.01, class_weight=None, dual=True, loss=hinge, max_iter=1000, penalty=l1, tol=0.0001; total time=   0.0s
[CV] END C=0.01, class_weight=None, dual=True, loss=hinge, max_iter=1000, penalty=l1, tol=0.001; total time=   0.0s
[CV] 

/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END C=10, class_weight=None, dual=True, loss=squared_hinge, max_iter=2000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=10, class_weight=None, dual=True, loss=squared_hinge, max_iter=2000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=10, class_weight=None, dual=True, loss=squared_hinge, max_iter=2000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=1, class_weight=None, dual=True, loss=squared_hinge, max_iter=3000, penalty=l2, tol=0.001; total time=   0.2s
[CV] END C=10, class_weight=None, dual=True, loss=squared_hinge, max_iter=2000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=10, class_weight=None, dual=True, loss=squared_hinge, max_iter=2000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=10, class_weight=None, dual=True, loss=squared_hinge, max_iter=2000, penalty=l1, tol=0.0001; total time=   0.0s
[CV] END C=10, class_weight=None, dual=True, loss=squared_hinge, max_iter=2000, penalty=l1, tol=0.0001; total time=   0.0s
[CV] END C=10, class_we

/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END C=10, class_weight=None, dual=False, loss=hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=10, class_weight=None, dual=False, loss=hinge, max_iter=2000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=10, class_weight=None, dual=False, loss=hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=10, class_weight=None, dual=False, loss=hinge, max_iter=2000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=10, class_weight=None, dual=False, loss=hinge, max_iter=2000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=10, class_weight=None, dual=False, loss=hinge, max_iter=2000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=10, class_weight=None, dual=False, loss=hinge, max_iter=3000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=10, class_weight=None, dual=False, loss=hinge, max_iter=3000, penalty=l1, tol=0.001; total time=   0.0s
[CV] END C=10, class_weight=None, dual=False, loss=hinge, max_iter=3000, penalty

...

/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END C=100, class_weight=balanced, dual=False, loss=squared_hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=100, class_weight=balanced, dual=False, loss=squared_hinge, max_iter=3000, penalty=l2, tol=0.0001; total time=   0.0s
[CV] END C=100, class_weight=balanced, dual=False, loss=squared_hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=100, class_weight=balanced, dual=False, loss=squared_hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=100, class_weight=balanced, dual=False, loss=squared_hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=100, class_weight=balanced, dual=False, loss=squared_hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END C=100, class_weight=balanced, dual=True, loss=squared_hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   1.3s
[CV] END C=100, class_weight=balanced, dual=True, loss=squared_hinge, max_iter=3000, penalty=l2, tol=1e-

/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1800 fits failed out of a total of 3600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-----------------------------------------------------

[CV] END C=100, class_weight=balanced, dual=True, loss=squared_hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   0.9s
[CV] END C=100, class_weight=balanced, dual=True, loss=squared_hinge, max_iter=3000, penalty=l2, tol=1e-05; total time=   0.9s


GridSearchCV(cv=5, estimator=LinearSVC(), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'class_weight': [None, 'balanced'],
                         'dual': [True, False],
                         'loss': ['hinge', 'squared_hinge'],
                         'max_iter': [1000, 2000, 3000],
                         'penalty': ['l1', 'l2'],
                         'tol': [0.001, 0.0001, 1e-05]},
             verbose=2)

In [38]:
print(svc_grid_search.best_params_)
print(svc_grid_search.best_estimator_)
print(svc_grid_search.score(X_test,y_test))

print(svc_grid_search.predict([[42,0,0,0,0,0,2,0,0,3,0,0,0]]))

{'C': 0.01, 'class_weight': 'balanced', 'dual': True, 'loss': 'hinge', 'max_iter': 1000, 'penalty': 'l2', 'tol': 0.001}
LinearSVC(C=0.01, class_weight='balanced', loss='hinge', tol=0.001)
0.8833107191316146


/Users/levi_1/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


ValueError: X has 13 features, but LinearSVC is expecting 16 features as input.